# XMILE => DEVSML

In [247]:
from jinja2 import DictLoader, Environment, FileSystemLoader
import os
PATH = './'
TEMPLATE_ENVIRONMENT = Environment(
    autoescape=False,
    loader=FileSystemLoader(os.path.join(PATH, 'templates')),
    trim_blocks=False)
def render_template(template_filename, context):
    return TEMPLATE_ENVIRONMENT.get_template(template_filename).render(context)
###################################################################################################################

import xml.etree.ElementTree as etree
source_xmlns = "{http://docs.oasis-open.org/xmile/ns/XMILE/v1.0}" 
source_xmlns_isee = "{http://iseesystems.com/XMILE}"
###################################################################################################################
def format_variables(model):
    model_name      = model.get('name')
    if model_name is None:
        model_name = 'top'
    variables       = model.find(source_xmlns + 'variables')
    modules         = variables.findall(source_xmlns + 'module')
    stocks          = variables.findall(source_xmlns + 'stock')
    auxs_and_ctes   = variables.findall(source_xmlns + 'aux')
    flows           = variables.findall(source_xmlns + 'flow')
    dependencies    = variables.find(source_xmlns_isee + 'dependencies')
    dependencies    = dependencies.findall(source_xmlns + 'var') if dependencies is not None else [] 
    dependencies_ = {}
    def get_params(eqn):
        import re
        text = eqn.replace(' ', '').replace('.','&')
        res = filter(lambda x : x != '', list(map(lambda x : x.replace('&', '.'), re.split("[()-/*]+", text))))
        return res

    for dep in dependencies:
        name = dep.get('name')
        inputs = dep.findall(source_xmlns + 'in')
        inputs_ = []
        for in_dep in inputs:
            inputs_.append(in_dep.text)
        dependencies_.update({name : inputs_})

    auxs_ = {}
    vars_with_dependencies = dependencies_.keys()
    for var in auxs_and_ctes:
        name = var.get('name')
        eqn  = var.find(source_xmlns + 'eqn').text
        if name in vars_with_dependencies:
            auxs_.update({name : {'eqn' : eqn, 'eqn_params' : get_params(eqn)}})
        else:
            auxs_.update({name : {'eqn' : eqn, 'eqn_params' : get_params(eqn)}})

    stocks_ = {}
    for stock in stocks:
        name = stock.get('name')
        eqn  = stock.find(source_xmlns + 'eqn').text
        inflows  = list(map(lambda x : x.text, stock.findall(source_xmlns + 'inflow')))
        outflows = list(map(lambda x : x.text, stock.findall(source_xmlns + 'outflow')))
        non_negative = True if stock.find(source_xmlns + 'non_negative') is not None else False
        stocks_.update({name : {
            'eqn' : eqn, 
            'inflows' : inflows, 
            'outflows' : outflows,
            'non_negative' : non_negative
        }})

    flows_ = {}
    for flow in flows:
        name = flow.get('name')
        eqn  = flow.find(source_xmlns + 'eqn').text
        flows_.update({name : {'eqn' : eqn, 'eqn_params' : get_params(eqn)}})

    modules_ = {}
    for module in modules:
        name = module.get('name')
        connections = module.findall(source_xmlns + 'connect')
        connections_ = []
        for connection in connections:
            to_   = connection.get('to')
            from_ = connection.get('from')
            connections_.append({'to' : to_, 'from' : from_})
        modules_.update({name : {'connections' : connections_}})
        
    return {
        'model_name' : model_name, 
        'modules' : modules_,
        'dependencies' : dependencies_,
        'stocks' : stocks_,
        'flows' : flows_,
        'auxs'  : auxs_
    }

###################################################################################################################
def generateDEVSML(formatted_variables, dst_xml, recursivity_level):
    
    model_name = formatted_variables['model_name']
    modules = formatted_variables['modules']
    auxs    = formatted_variables['auxs']
    flows   = formatted_variables['flows']
    stocks  = formatted_variables['stocks']
    dependencies = formatted_variables['dependencies']

    all_variables = stocks.keys() + flows.keys() + auxs.keys()
    
    TEMPLATE_COUPLED_DEVSML = 'template-coupled-devsml.xml'
    
    atomics = []
    coupled_atomics = []
    
    internal_connections = []
    coupled_internal_connections = []
    
    external_input_connections = []
    coupled_external_input_connections = []
    
    external_output_connections = []
    coupled_external_output_connections = []

    #####################
    # Atomicos (parametros de input para el top model)
    for auxName, auxAttr in auxs.iteritems():
        eqn_params = filter(lambda x : x in all_variables, auxAttr['eqn_params'])

        atomics.append({
            'name' : auxName,
            'model' : 'Cte' if eqn_params == [] else 'Aux',
            'in_ports' : [{'name' : elem, 'type' : 'in'} for elem in eqn_params],
            'out_ports' : [{'name' : 'out', 'type' : 'out'}],
            'parameters' : [
                {'name' : 'function', 'function' : auxAttr['eqn']}
            ]
        })
    
    #####################
    # VER ESTO !!!
    # Acoplado (un unico acoplado que contiene a todo lo que no son input parameters del modelo)
    for stockName, stockAttr in stocks.iteritems():
        coupled_atomics.append({
            'name' : stockName + 'Tot',
            'model' : 'Ftot',
            'in_ports' : [{'name' : elem + 'Plus' , 'type' : 'in_plus'} for elem in stockAttr['inflows']] +
                         [{'name' : elem + 'Minus', 'type' : 'in_minus'} for elem in stockAttr['outflows']],
            'out_ports' : [{'name' :'out', 'type' : 'out'}]
        })
        coupled_atomics.append({
            'name' : stockName,
            'model' : 'QSS1',
            'in_ports' : [{'name' : 'in', 'type' : 'in'}],
            'out_ports' : [{'name' : 'out', 'type' : 'out'}],
            'parameters' : [
                {'name' : 'x0', 'function' : stockAttr['eqn']},
                {'name' : 'dQRel', 'function' : 1e-4},
                {'name' : 'dQMin', 'function' : 1e-4}
            ]
        })

    for flowName, flowAttr in flows.iteritems():
        coupled_atomics.append({
            'name' : flowName + 'Plus',
            'model' : 'Fpm',
            'in_ports' : [{'name' : elem, 'type' : 'in'} for elem in
                          filter(lambda x : x in all_variables, flowAttr['eqn_params'])],
            'out_ports' : [{'name' : 'out', 'type' : 'out'}],
            'parameters' : [
                {'name' : 'function', 'function' : flowAttr['eqn']}
            ]
        })
        coupled_atomics.append({
            'name' : flowName + 'Minus',
            'model' : 'Fpm',
            'in_ports' : [{'name' : elem, 'type' : 'in'} for elem in
                          filter(lambda x : x in all_variables, flowAttr['eqn_params'])],
            'out_ports' : [{'name' : 'out', 'type' : 'out'}],
            'parameters' : [
                {'name' : 'function', 'function' : flowAttr['eqn']}
            ]
        })

    #####################
    # Conexiones internas

    # Ftot => Integrador
    for stockName, stockAttr in stocks.iteritems():
        coupled_internal_connections.append({
            'component_from' : stockName + 'Tot',
            'component_to' : stockName,
            'port_from' : 'out',
            'port_to' : 'in'
        })
    # Fpm => Ftot
    for flowName, flowAttr in flows.iteritems():
        for stockName, stockAttr in stocks.iteritems():
            if flowName in stockAttr['inflows']:
                coupled_internal_connections.append({
                    'component_from' : flowName + 'Plus',
                    'component_to' : stockName + 'Tot',
                    'port_from' : 'out',
                    'port_to' : flowName + 'Plus'
                })
            if flowName in stockAttr['outflows']:
                coupled_internal_connections.append({
                    'component_from' : flowName + 'Minus',
                    'component_to' : stockName + 'Tot',
                    'port_from' : 'out',
                    'port_to' : flowName + 'Minus'
                })
    # Stock => {Flow, Aux}
    for stockName, stockAttr in stocks.iteritems():
        # Flow
        for flowName, flowAttr in flows.iteritems():
            if stockName in flowAttr['eqn_params']:# and flowName in stockAttr['inflows']:
                coupled_internal_connections.append({
                    'component_from' : stockName,
                    'component_to' : flowName + 'Plus',
                    'port_from' : 'out',
                    'port_to' : stockName
                })
            if stockName in flowAttr['eqn_params']:# and flowName in stockAttr['outflows']:
                coupled_internal_connections.append({
                    'component_from' : stockName,
                    'component_to' : flowName + 'Minus',
                    'port_from' : 'out',
                    'port_to' : stockName
                })
        # Aux
        for auxName, auxAttr in auxs.iteritems():
            if stockName in auxAttr['eqn_params']:
                coupled_internal_connections.append({
                    'component_from' : stockName,
                    'component_to' : auxName,
                    'port_from' : 'out',
                    'port_to' : stockName
                })
    # Aux => {Aux, Flow}
    for auxName, auxAttr in auxs.iteritems():
        # Flow
        for flowName, flowAttr in flows.iteritems():
            if auxName in flowAttr['eqn_params']:
                
                # TODO : ver si esto alcanza para considerar el 'recursivity_level'
                port_from = auxName
                
                coupled_external_input_connections.append({
                    'component_from' : auxName,
                    'component_to' : flowName + 'Plus',
                    'port_from' : port_from,
                    'port_to' : auxName
                })
                coupled_external_input_connections.append({
                    'component_from' : auxName,
                    'component_to' : flowName + 'Minus',
                    'port_from' : port_from,
                    'port_to' : auxName
                })
        # Aux
        for auxName2, auxAttr2 in auxs.iteritems():
            if auxName in auxAttr2['eqn_params'] and auxName != auxName2:
                
                # TODO : ver si esto alcanza para considerar el 'recursivity_level'
                port_from = auxName
                
                coupled_external_input_connections.append({
                    'component_from' : auxName,
                    'component_to' : auxName2,
                    'port_from' : port_from,
                    'port_to' : auxName
                })

    #####################
    # Input Ports : los Aux's que no tienen valor, o que tienen un valor numerico
    in_ports = []
    for auxName, auxAttr in auxs.iteritems():
        # NOTA : los que estan vacios ASUMO que es porque viene de un modelo acoplado de afuera
        emptyEqn = (auxAttr['eqn'] == '')
        if emptyEqn or all([not elem in all_variables for elem in auxAttr['eqn_params']]):
            in_ports.append(auxName)
    coupled_in_ports = in_ports

    # Output Ports : en ppio., todos los stocks
    out_ports = []
    for stockName, stockAttr in stocks.iteritems():
        out_ports.append(stockName)
    coupled_out_ports = out_ports
        
    #####################
    # Conexiones externas de input : cte's (desde el punto de vista de este acoplado)
    for auxName, auxAttr in auxs.iteritems():
        eqn_params = filter(lambda x : x in all_variables, auxAttr['eqn_params'])
        if eqn_params == []: # model == 'Cte'
            external_input_connections.append({
                'port_from' : auxName,
                'port_to' : 'inValue',
                'component_to' : auxName
            })
            
    #####################
    # Conexiones externas de output (no-coupled) : stocks
    for stockName, stockAttr in stocks.iteritems():
        external_output_connections.append({
            'port_from' : stockName,
            'component_from' : 'coupled_level_' + str(recursivity_level + 1),
            'port_to' : stockName
        })
            
    #####################
    # Conexiones externas de output (coupled) : stocks
    for stockName, stockAttr in stocks.iteritems():
        coupled_external_output_connections.append({
            'port_from' : 'out',
            'component_from' : stockName,
            'port_to' : stockName
        })
        
    #####################
    # Conexiones internas (no-coupled) (conexiones entre atomicos y entre atomics y coupled)
    # atomicos <-> coupled
    for auxName, auxAttr in auxs.iteritems():
        eqn_params = filter(lambda x : x in all_variables, auxAttr['eqn_params'])
        if eqn_params == []: # model == 'Cte'
            internal_connections.append({
                'component_from' : auxName,
                'port_from' : auxName,
                'port_to' : auxName,
                'component_to' : 'coupled_level_' + str(recursivity_level + 1)
            })
    
    # atomicos <-> atomicos
    # TODO (en general no va a pasar)
    
    ###############################################################
    coupled_filenames = [
        dst_xml + '_' + moduleName + '_coupled' + '.xml'
        for moduleName in modules.keys()
    ]
    
    #### Genero el XML para cada uno de los ACOPLADOS de este modelo ####
    # generateDEVSML(formatted_variables, dst_xml, recursivity_level)
    context_coupleds = {
        'coupled_model_name'         : model_name + '_atomic_' + str(recursivity_level),
        'in_ports'                   : in_ports,
        'out_ports'                  : out_ports,
        'atomics'                    : atomics,
        'coupled_filenames'          : coupled_filenames,
        'internal_connections'       : internal_connections,
        'external_input_connections' : external_input_connections,
        'external_output_connections': external_output_connections
    }
    with open(dst_xml + '_' + model_name + '_atomic' + '.xml', 'w') as f1:
        f1.write(render_template(TEMPLATE_COUPLED_DEVSML,context_coupleds))
    
    # TODO : aca deberia ir la recursion ( generar los XML para los modulos que hay en cada acoplado )
    # NOTA : son los coupled_{module name}_{recursivity level}.xml que se van a leer desde el template
    # generar 'coupled/coupled_PredatorModel_1.xml', por ejemplo
    ##
    
    # TODO : ahora deberia combinar los atomicos y los acoplados de este modelo
    #if len(coupled_filenames) > 0:
    context = {
        'coupled_model_name'          : model_name + '_coupled_' + str(recursivity_level),
        'in_ports'                    : coupled_in_ports,
        'out_ports'                   : coupled_out_ports,
        'atomics'                     : coupled_atomics,
        'coupled_filenames'           : coupled_filenames, # TODO : descomentar esto!
        'internal_connections'        : coupled_internal_connections,
        'external_input_connections'  : coupled_external_input_connections,
        'external_output_connections' : coupled_external_output_connections
    }
    with open(dst_xml + '_' + model_name + '_coupled' '.xml', 'w') as f:
        f.write(render_template(TEMPLATE_COUPLED_DEVSML,context))
    
    #   
    return True
#################################################################################################################

# XMILE Nested

In [251]:
DIR_XMILE       = 'lotka-volterra-nested/lotka-volterra-nested-2.xmile'
DIR_DEVSML_BASE = 'lotka-volterra-nested/lvn'#.xml'

###################################################################################################################
parser = etree.XMLParser(encoding="utf-8")
with open(DIR_XMILE, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()

header    = root.find(source_xmlns + 'model')
sim_specs = root.find(source_xmlns + 'sim_specs')
start     = sim_specs.find(source_xmlns + 'start').text
stop      = sim_specs.find(source_xmlns + 'stop').text
prefs     = root.find(source_xmlns_isee + 'prefs')
multiplayer_settings = root.find(source_xmlns_isee + 'multiplayer_settings')
model_units          = root.find(source_xmlns + 'model_units')

models = root.findall(source_xmlns + 'model')
models_formatted_variables = {}
for model in models:
    if model.get('name') is None:
        models_formatted_variables.update({'top' : format_variables(model)})
    else:
        models_formatted_variables.update({model.get('name') : format_variables(model)})

# Asumo que el modelo que no tiene nombre es el de mayor abstraccion
def traverse(parent, models_map, level):
    top = models_map[parent]
    generateDEVSML(top, DIR_DEVSML_BASE, level)
    children = top['modules']
    for child in children:
        traverse(child, models_map, level + 1)
        
traverse('top', models_formatted_variables, 0)        


In [ ]:
#1 - Correr 'generateDEVSML' para cada uno de los modelos que hayan en el XMILE
#2 - Correr una version modificada de 'generateDEVSML' para el top model del modelo XMILE

# XMILE No Nested

In [133]:
DIR_XMILE  = 'lotka-volterra/lotka-volterra.xmile'
DIR_DEVSML = 'lotka-volterra/lotka-volterra-devsml-traduccion.xml'

###################################################################################################################
parser = etree.XMLParser(encoding="utf-8")
with open(DIR_XMILE, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()

header = root.find(source_xmlns + 'model')
sim_specs = root.find(source_xmlns + 'sim_specs')
start = sim_specs.find(source_xmlns + 'start').text
stop  = sim_specs.find(source_xmlns + 'stop').text

prefs = root.find(source_xmlns_isee + 'prefs')
multiplayer_settings = root.find(source_xmlns_isee + 'multiplayer_settings')
model_units = root.find(source_xmlns + 'model_units')
model = root.find(source_xmlns + 'model')

formatted_variables = format_variables(model)
generateDEVSML(formatted_variables, DIR_DEVSML, 0)

True

In [253]:
from py_expression_eval import Parser
parser = Parser()

def get_params(eqn):
        import re
        text = eqn.replace(' ', '').replace('.','&')
        res = filter(lambda x : x != '', list(map(lambda x : x.replace('&', '.'), re.split("[()-/*]+", text))))
        return res

In [256]:
eqn1 = 'var1 + var2 / 3'
exp1 = parser.parse(eqn1)
exp1.variables()

['var1', 'var2']